In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os 
import glob
import h5py
import time
import numpy as np
import pprint

In [2]:
def load_data(is_train):
    if is_train:
        data_dir1 = os.path.join(os.getcwd(), 'train_set', "LR0") # Join the Train dir to current directory
        data1 = glob.glob(os.path.join(data_dir1, "*.jpg")) # make set of all dataset file path
        print(data_dir1, data1[0:5])
        data_dir2 = os.path.join(os.getcwd(), 'train_set', "HR0") # Join the Train dir to current directory
        data2 = glob.glob(os.path.join(data_dir2, "*.jpg")) # make set of all dataset file path
    return data1,data2


def make_sub_data(data1,data2, padding, config):
    """
        Make the sub_data set
        Args:
            data : the set of all file path 
            padding : the image padding of input to label
            config : the all flags
    """
    sub_input_sequence = []
    sub_label_sequence = []
    for i in range(len(data1)):
        if config.is_train:
            input_=data1[i]
            label_=data2[i]
            
            
            input_=cv2.imread(input_)
            input_=cv2.resize(input_,None,fx = 2 ,fy = 2, interpolation = cv2.INTER_CUBIC)
            label_=cv2.imread(label_)
            
        if len(input_.shape) == 3: # is color
            h, w, c = input_.shape
        else:
            h, w = input_.shape # is grayscale
 
        for x in range(0, h - config.image_size + 1, config.stride):
            for y in range(0, w - config.image_size + 1, config.stride):

                sub_input = input_[x: x + config.image_size, y: y + config.image_size] # 33 * 33
                sub_label = label_[x + padding: x + padding + config.label_size, y + padding: y + padding + config.label_size] # 21 * 21


                # Reshape the subinput and sublabel
                sub_input = sub_input.reshape([config.image_size, config.image_size, config.c_dim])
                sub_label = sub_label.reshape([config.label_size, config.label_size, config.c_dim])
                # Normialize
                sub_input =  sub_input / 255.0
                sub_label =  sub_label / 255.0
                
                #cv2.imshow("im1",sub_input)
                #cv2.imshow("im2",sub_label)
                #cv2.waitKey(0)

                # Add to sequence
                sub_input_sequence.append(sub_input)
                sub_label_sequence.append(sub_label)

    return sub_input_sequence, sub_label_sequence

def make_data_hf(input_, label_, config):
    """
        Make input data as h5 file format
        Depending on "is_train" (flag value), savepath would be change.
    """
    # Check the check dir, if not, create one
    if not os.path.isdir(os.path.join(os.getcwd(),config.checkpoint_dir)):
        os.makedirs(os.path.join(os.getcwd(),config.checkpoint_dir))

    if config.is_train:
        savepath = os.path.join(os.getcwd(), config.checkpoint_dir + '/train.h5')

    with h5py.File(savepath, 'w') as hf:
        hf.create_dataset('input', data=input_)
        hf.create_dataset('label', data=label_)

def input_setup(config):
    """
        Read image files and make their sub-images and saved them as a h5 file format
    """

    # Load data path, if is_train False, get test data
    data1,data2 = load_data(config.is_train)

    padding = abs(config.image_size - config.label_size) // 2

    # Make sub_input and sub_label
    sub_input_sequence, sub_label_sequence = make_sub_data(data1,data2, padding, config)


    # Make list to numpy array. With this transform
    arrinput = np.asarray(sub_input_sequence) # [?, 33, 33, 3]
    arrlabel = np.asarray(sub_label_sequence) # [?, 21, 21, 3]

    make_data_hf(arrinput, arrlabel, config)



##############################################################################



def checkpoint_dir(config):
    if config.is_train:
        return os.path.join('./{}'.format(config.checkpoint_dir), "train.h5")
    #else:
        #return os.path.join('./{}'.format(config.checkpoint_dir), "test.h5")
    
    



#########################################################################################

def read_data(path):
    """
        Read h5 format data file

        Args:
            path: file path of desired file
            data: '.h5' file format that contains  input values
            label: '.h5' file format that contains label values 
    """
    with h5py.File(path, 'r') as hf:
        input_ = np.array(hf.get('input'))
        label_ = np.array(hf.get('label'))
        return input_, label_


In [3]:
class SRCNN(object):

    def __init__(self,sess, image_size, label_size,c_dim):
        self.sess = sess
        self.image_size = image_size
        self.label_size = label_size
        self.c_dim = c_dim
        self.build_model()
        
    def train(self, config):
        input_setup(config)

        data_dir = checkpoint_dir(config)

        input_, label_ = read_data(data_dir)
        print(input_)
        #print(input_)
        # Stochastic gradient descent with the standard backpropagation
        #self.train_op = tf.train.GradientDescentOptimizer(config.learning_rate).minimize(self.loss)
        self.train_op = tf.train.AdamOptimizer(learning_rate=config.learning_rate).minimize(self.loss)
        tf.global_variables_initializer().run()
        counter = 0
        time_ = time.time()

        # Train
        if config.is_train:
            print("Now Start Training...")
            for ep in range(config.epoch):
                # Run by batch images
                batch_idxs = len(input_) // config.batch_size
                for idx in range(0, batch_idxs):
                    batch_images = input_[idx * config.batch_size : (idx + 1) * config.batch_size]
                    batch_labels = label_[idx * config.batch_size : (idx + 1) * config.batch_size]
                    counter += 1
                    _, err = self.sess.run([self.train_op, self.loss], feed_dict={self.images: batch_images, self.labels: batch_labels})

                    if counter % 10 == 0:
                        print("Epoch: [%2d], step: [%2d], time: [%4.4f], loss: [%.8f]" % ((ep+1), counter, time.time()-time_, err))
                        #print(label_[1] - self.pred.eval({self.images: input_})[1],'loss:]',err)
                    if counter % 10 == 0:
                        self.save(config.checkpoint_dir, counter)

                            
    def build_model(self):
        self.images = tf.placeholder(tf.float32, [None, self.image_size, self.image_size, self.c_dim], name='images')
        self.labels = tf.placeholder(tf.float32, [None, self.label_size, self.label_size, self.c_dim], name='labels')
        
        self.weights = {
            'w1': tf.Variable(tf.random_normal([9, 9, self.c_dim, 64], stddev=1e-3), name='w1'),
            'w2': tf.Variable(tf.random_normal([1, 1, 64, 32], stddev=1e-3), name='w2'),
            'w3': tf.Variable(tf.random_normal([5, 5, 32, self.c_dim], stddev=1e-3), name='w3')
        }

        self.biases = {
            'b1': tf.Variable(tf.zeros([64], name='b1')),
            'b2': tf.Variable(tf.zeros([32], name='b2')),
            'b3': tf.Variable(tf.zeros([self.c_dim], name='b3'))
        }
        
        self.pred = self.model()
        
        self.loss = tf.reduce_mean(tf.square(self.labels - self.pred))

        self.saver = tf.train.Saver() # To save checkpoint

    def model(self):
        conv1 = tf.nn.relu(tf.nn.conv2d(self.images, self.weights['w1'], strides=[1,1,1,1], padding='VALID') + self.biases['b1'])
        conv2 = tf.nn.relu(tf.nn.conv2d(conv1, self.weights['w2'], strides=[1,1,1,1], padding='VALID') + self.biases['b2'])
        conv3 = tf.nn.conv2d(conv2, self.weights['w3'], strides=[1,1,1,1], padding='VALID') + self.biases['b3'] # This layer don't need ReLU
        return conv3

    
    
    def save(self, checkpoint_dir, step):
        """
            To save the checkpoint use to test or pretrain
        """
        model_name = "SRCNN.model"
        model_dir = "%s_%s" % ("srcnn", self.label_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
             os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, model_name),global_step=step)

In [4]:
class this_config():
    def __init__(self, is_train=True):
        self.epoch = 1
        self.image_size = 33
        self.label_size = 21
        self.c_dim = 3
        self.is_train = is_train
        self.scale = 3
        self.stride = 21
        self.checkpoint_dir = "checkpoint"
        self.learning_rate = 1e-4
        self.batch_size = 128
        
arg = this_config()
print("Hello TA!  We are group 7. Thank you for your work for us. Hope you have a happy day!")

with tf.Session() as sess:
    FLAGS = arg
    srcnn = SRCNN(sess,
                  image_size = FLAGS.image_size,
                  label_size = FLAGS.label_size,
                  c_dim = FLAGS.c_dim)
    srcnn.train(FLAGS)

Hello TA!  We are group 7. Thank you for your work for us. Hope you have a happy day!
/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0 ['/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0/img_0014.jpg', '/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0/img_0028.jpg', '/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0/img_0029.jpg', '/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0/img_0001.jpg', '/Users/yaoqi/Google Drive/TensorFlow-SRCNN-new/train_set/LR0/img_0015.jpg']
[[[[0.50588235 0.58431373 0.62352941]
   [0.46666667 0.54901961 0.59607843]
   [0.40392157 0.49019608 0.54117647]
   ...
   [0.1254902  0.13333333 0.10196078]
   [0.12941176 0.14117647 0.10980392]
   [0.13333333 0.14901961 0.1254902 ]]

  [[0.37647059 0.45098039 0.49411765]
   [0.36862745 0.44705882 0.49411765]
   [0.35294118 0.43529412 0.49019608]
   ...
   [0.12156863 0.1372549  0.10196078]
   [0.12156863 0.1372549  0.10588235]
   [0.12156863 0.14117647 0.11764

Now Start Training...
Epoch: [ 1], step: [10], time: [5.2914], loss: [0.26741955]
Epoch: [ 1], step: [20], time: [9.8307], loss: [0.12420551]
Epoch: [ 1], step: [30], time: [13.9746], loss: [0.19889694]
Epoch: [ 1], step: [40], time: [18.4197], loss: [0.14837749]
Epoch: [ 1], step: [50], time: [23.1074], loss: [0.13176635]
Epoch: [ 1], step: [60], time: [26.9377], loss: [0.01586235]
Epoch: [ 1], step: [70], time: [30.5182], loss: [0.02361333]
Epoch: [ 1], step: [80], time: [35.6888], loss: [0.00288014]
Epoch: [ 1], step: [90], time: [39.6543], loss: [0.00716053]
Epoch: [ 1], step: [100], time: [43.5741], loss: [0.00931056]
Epoch: [ 1], step: [110], time: [47.0611], loss: [0.00554266]
Epoch: [ 1], step: [120], time: [51.2929], loss: [0.01047839]
Epoch: [ 1], step: [130], time: [54.4445], loss: [0.00936430]
Epoch: [ 1], step: [140], time: [57.6443], loss: [0.00664726]
Epoch: [ 1], step: [150], time: [60.8237], loss: [0.01643269]
Epoch: [ 1], step: [160], time: [64.2845], loss: [0.0411756